# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
experiment='cxix40218'
runNumber = 50

# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *
# Determine current user
currentUser, error = unixCMD("echo $USER")
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsTools')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

# Load timebinned CSPAD

In [ ]:
timebins = np.load(os.environ['RESULTSPATH']+'/timebins-run-%d.npy' % runNumber )
CSPAD = np.load(os.environ['RESULTSPATH']+'/CSPAD-run-%d.npy' % runNumber)
variance = np.load(os.environ['RESULTSPATH']+'/variance-run-%d.npy' % runNumber)
counts = np.load(os.environ['RESULTSPATH']+'/counts-run-%d.npy' % runNumber)


# Plot CSPAD

In [ ]:
from IPython.display import clear_output
from plotStyles import *

def plotCSPAD( cspad , x , y, cspadMask=None, zLims = None, divergent=False, NTILE=32 ):
    figOpts = {'xLims':[-2e5,2e5],'yLims':[-2e5,2e5],'divergent':divergent, 'xIn':3, 'yIn':3*11.5/14.5}
    
    if zLims is not None:
        figOpts['zLims'] = zLims
    
    for iTile in range(NTILE):
    
        if cspadMask is not None:
            cspadTile = cspad[iTile,:,:]
            tileMask = ~cspadMask[iTile,:,:]
            cspadTile[tileMask] = 0
        
        if iTile == 0:
            newFigure = True
        else:
            newFigure = False
            
        clear_output()
        colorPlot( x[iTile,:,:], y[iTile,:,:], cspadTile , newFigure=newFigure, **figOpts);


x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)
# cspadMask = createMask(detType='Jungfrau').astype(bool)
cspadMask = np.ones_like(x).astype(bool)
print(cspadMask.shape)


CSPADbinned = CSPAD
        
# plotCSPAD( cspadMask, x , y , cspadMask=cspadMask, divergent=True )
plotCSPAD( CSPADbinned[:,:,:,5]-CSPADbinned[:,:,:,0], x , y , cspadMask=cspadMask, divergent=True, NTILE=8 )



# Isotropic analysis

In [ ]:
# Read in geometry and geometry measure dduring beamtime
# XC,YC = 88097, 93696
XC,YC = 0, 0
X=np.array(x-XC)
Y=np.array(y-YC)
R=np.sqrt(np.power(X,2)+np.power(Y,2))
L=69e3

# Use two-theta to calculate Q
TwoTheta=np.arctan(R/L)
Q = 4*np.pi*np.sin(TwoTheta/2)/1.3874;
print np.max(Q)
PHID = np.arctan2(X,Y)#Y,X)
THETAD = TwoTheta;

# Form the Thomson correction factor
# THOMSON_CORRECTION = PHID; 
THOMSON_CORRECTION = ((np.sin(PHID)**2+np.cos(THETAD)**2.*np.cos(PHID)**2)); 

# Radial integration

In [ ]:
def radially_integrate(rgrid,Rmap,image,Mask=None):
    dr = rgrid[2]-rgrid[1]
    rsum=np.zeros_like(rgrid)
    
    for idx, r in enumerate(rgrid):
        rlower = r - dr/2.
        rupper = r + dr/2.

        if Mask is None:
            little_slice_r=((Rmap > rlower)&(Rmap < rupper))    
        else:
            little_slice_r=~((~((Rmap > rlower)&(Rmap < rupper))) | (Mask))

        normalizer=(np.nansum(np.abs(image[little_slice_r])>0))    
        if normalizer==0:
            rsum[idx]=np.nan
        else:
            rsum[idx] = np.nansum(image[little_slice_r]) / normalizer
    return rsum, normalizer

k0 = 4.5287 # 2pi/lambda

NQ = 200 # Must be divisible by 2
QMIN = 0.
QMAX = k0
qgrid = np.linspace(QMIN,QMAX,NQ)

NT = timebins.size
rad_int = np.zeros((NQ,NT))
rad_std = np.zeros((NQ,NT))

for t_idx in xrange(NT):
    print (t_idx, NT)
    rad_int[:,t_idx],nsum=radially_integrate(qgrid,Rmap=Q, image=CSPAD[:,:,:,t_idx]/THOMSON_CORRECTION, Mask=(cspadMask<1))
    var,nsum=radially_integrate(qgrid,Rmap=Q, image=variance[:,:,:,t_idx]/(THOMSON_CORRECTION**2), Mask=(cspadMask<1))
    
    rad_std[:,t_idx]=np.sqrt(var/nsum)

In [ ]:
iT0=9
iQ0=20


TT,QQ = np.meshgrid(timebins[iT0:], qgrid[iQ0:])


plotme = (rad_int[iQ0:,iT0:])
plotme = plotme - plotme[:,0][:,np.newaxis]
plotme = plotme
scaleFac = plotme.flatten().max()
plotme = plotme / scaleFac

stdt0 = rad_std[iQ0:,iT0]


figOpts = {'xLims':[1,4.],'yLims':[0,2.5],'zLims':[-1,1], 'yLabel':r'$\tau$', 'yUnits':'ps', 'xLabel':'Q', 'xUnits':r'$\AA^{-1}$',
           'nxTicks':4, 'nyTicks':6, 'zLabel':r'$S_0(Q,\tau)-S_{u}(Q)$', 'divergent':True}

colorPlot(  qgrid[iQ0:],timebins[iT0:]-2.35,  (plotme).T , newFigure=True, **figOpts)

plt.ylim([-.05,2.5])
plt.xlim([1,4.5])


figOpts = {'xLims':[1,4.],'yLims':[0,2.5], 'yLabel':r'$\tau$', 'yUnits':'ps', 'xLabel':'Q', 'xUnits':r'$\AA^{-1}$',
           'nxTicks':4, 'nyTicks':6, 'zLabel':r'$STD$', 'divergent':False}
colorPlot( qgrid[iQ0:], timebins[iT0:]-2.35,   (( rad_std[iQ0:,iT0:]+stdt0[:,np.newaxis] )/scaleFac).T, newFigure=True, **figOpts)



